In [104]:
import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
import statistics

In [105]:
data_dir = '/Users/anishkahc/Desktop/Docs/ML Fall 23/Data/'

openap_file = 'OpenAP_Macro.parquet.gzip'
openap = pd.read_parquet( data_dir + openap_file)

In [106]:
openap.head()

,permno,DateYM,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AM,...,Rfree,macro_tbl,macro_svar,macro_ntis,macro_bm,macro_dp,macro_ep,macro_tms,macro_dfy,retadj
0,10000,1986-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0056,0.0707,0.001920,-0.019172,0.583517,-3.283635,-2.675897,0.0251,0.0139,-0.257143
1,10000,1986-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0053,0.0706,0.001089,-0.017914,0.536377,-3.347659,-2.747007,0.0135,0.0144,0.365385
2,10000,1986-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0060,0.0656,0.001374,-0.016420,0.519628,-3.394107,-2.800518,0.0110,0.0150,-0.098592
3,10000,1986-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0052,0.0606,0.002459,-0.024585,0.529714,-3.376538,-2.781919,0.0176,0.0140,-0.222656
4,10000,1986-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0049,0.0615,0.001370,-0.021872,0.503541,-3.422238,-2.826589,0.0233,0.0120,-0.005025


In [107]:
openap = openap[pd.DatetimeIndex(openap['DateYM']).year >= 2000]

In [108]:
openap.drop(openap.iloc[:,-10:-1], inplace = True, axis = 1)

In [109]:
openap.set_index(keys = ['DateYM', 'permno'], inplace = True, verify_integrity = True)

In [110]:
#2 rank
cols = openap.columns[:-1]
openap[cols] = openap.groupby('DateYM')[cols].rank(pct=True).subtract(0.5).multiply(2)
openap.head()

,,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AM,AnalystRevision,AnalystValue,...,VolumeTrend,XFIN,zerotrade,zerotradeAlt1,zerotradeAlt12,mvel1,mom1m,baspread,sic2,retadj
DateYM,permno,,,,,,,,,,,,,,,,,,,,,
2000-01-31,10001,0.585548,0.732207,NaN,NaN,NaN,NaN,NaN,0.239803,-0.000846,-0.110184,...,-0.231566,0.587691,0.727203,0.755337,0.735880,-0.706514,-0.108218,-0.924625,-0.094187,-0.044118
2000-02-29,10001,0.587625,0.733617,NaN,NaN,NaN,NaN,NaN,0.278128,-0.061415,-0.112281,...,-0.184695,0.585773,0.725440,0.863022,0.734810,-0.738480,-0.266781,-0.670874,-0.094956,0.015385
2000-03-31,10001,0.591922,0.732450,NaN,NaN,NaN,NaN,NaN,0.278420,-0.226681,-0.111269,...,-0.097580,0.581733,0.773035,0.714462,0.720754,-0.757746,0.042914,-0.874097,-0.095455,-0.015758
2000-04-30,10001,0.593240,0.738564,NaN,NaN,NaN,NaN,NaN,0.298565,-0.045386,-0.108244,...,-0.038175,0.583394,0.815341,0.885350,0.726192,-0.760314,-0.086083,-0.819030,-0.093233,0.011719
2000-05-31,10001,0.593883,0.737595,NaN,NaN,NaN,NaN,NaN,0.271676,-0.055184,-0.104197,...,0.009154,0.581044,0.838397,0.884606,0.746687,-0.722998,0.394935,-0.818271,-0.090394,-0.023166


In [111]:
#3 missing values
def fill_missing_values(x):
    median_value = x.median()
    return x.fillna(0 if pd.isna(median_value) else median_value)

openap = openap.groupby('DateYM').transform(fill_missing_values)

In [112]:
openap.head()

,,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AM,AnalystRevision,AnalystValue,...,VolumeTrend,XFIN,zerotrade,zerotradeAlt1,zerotradeAlt12,mvel1,mom1m,baspread,sic2,retadj
DateYM,permno,,,,,,,,,,,,,,,,,,,,,
2000-01-31,10001,0.585548,0.732207,-0.421196,-0.038095,0.006024,0.000656,-0.008746,0.239803,-0.000846,-0.110184,...,-0.231566,0.587691,0.727203,0.755337,0.735880,-0.706514,-0.108218,-0.924625,-0.094187,-0.044118
2000-02-29,10001,0.587625,0.733617,-0.426273,-0.033386,0.006024,0.000660,0.029665,0.278128,-0.061415,-0.112281,...,-0.184695,0.585773,0.725440,0.863022,0.734810,-0.738480,-0.266781,-0.670874,-0.094956,0.015385
2000-03-31,10001,0.591922,0.732450,-0.410811,-0.030303,0.006135,0.000667,-0.012207,0.278420,-0.226681,-0.111269,...,-0.097580,0.581733,0.773035,0.714462,0.720754,-0.757746,0.042914,-0.874097,-0.095455,-0.015758
2000-04-30,10001,0.593240,0.738564,-0.400000,-0.014400,0.006135,0.000665,0.017992,0.298565,-0.045386,-0.108244,...,-0.038175,0.583394,0.815341,0.885350,0.726192,-0.760314,-0.086083,-0.819030,-0.093233,0.011719
2000-05-31,10001,0.593883,0.737595,-0.411924,-0.009662,0.006135,0.000667,0.024038,0.271676,-0.055184,-0.104197,...,0.009154,0.581044,0.838397,0.884606,0.746687,-0.722998,0.394935,-0.818271,-0.090394,-0.023166


In [113]:
#6 training, validation, test split
def split_val(openap):
    return model_selection.PredefinedSplit(openap['test_fold'])

In [114]:
#8 R^2 calculation
def R2_calc(y, y_pred):
    mse = 0
    mse_benchmark = 0
    for i in range(len(y)):
        mse += (y[i] - y_pred[i])**2
        mse_benchmark += y[i]**2
    return 1 - mse/mse_benchmark

In [115]:
year=[]
train_r2 = []
r2_result = []

train_start = 2000
train_end = 2005
validation_end = 2008
test_end = 2009

In [116]:
n = 10
    
for i in range(n):
    print("Training year ", i + 2009) 
    year.append(i + 2009)

    train_val = openap[(openap.index.get_level_values('DateYM').year >= train_start) 
                       & (openap.index.get_level_values('DateYM').year <= validation_end+i)].copy()
    test = openap[(openap.index.get_level_values('DateYM').year > validation_end+i) 
                  & (openap.index.get_level_values('DateYM').year <= test_end+i)].copy()
        
    train_val = train_val.assign(test_fold = -1)
    train_val.loc[pd.DatetimeIndex(train_val.index.get_level_values('DateYM')).year > train_end+i, 'test_fold'] = 0
        
    elastic = linear_model.ElasticNetCV(cv=split_val(train_val), n_jobs=-1)
    elastic.fit(X=train_val.drop(columns=['test_fold', 'retadj']), y=train_val['retadj'])
    
    y_hat = elastic.predict(X = test.drop(columns = ['retadj']))
    
    train_R2.append(R2_calc(train_val.loc[(train_val['test_fold'] == 0), 'retadj'], 
                           elastic.predict(train_val.loc[train_val['test_fold'] == 0].drop(columns = ['test_fold','retadj']))))
    
    r2_result.append(R2_calc(test['retadj'], y_hat))
    

    print('R2 for', i+6,'training years: ', str(r2_result[i]))
    print()
        

Training year  2009
R2 for 6 training years:  -0.010911524271817408

Training year  2010
R2 for 7 training years:  -0.010202959635886355

Training year  2011
R2 for 8 training years:  -0.0011939800160858383

Training year  2012
R2 for 9 training years:  0.008072090683067135

Training year  2013
R2 for 10 training years:  0.01999925968403049

Training year  2014
R2 for 11 training years:  -0.0022137137441342603

Training year  2015
R2 for 12 training years:  -0.0068892695045572605

Training year  2016
R2 for 13 training years:  0.005824573761483487

Training year  2017
R2 for 14 training years:  0.005404655461342722

Training year  2018
R2 for 15 training years:  -0.009529514326773958



In [118]:
print(year)

[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [120]:
print(r2_result)

[-0.010911524271817408, -0.010202959635886355, -0.0011939800160858383, 0.008072090683067135, 0.01999925968403049, -0.0022137137441342603, -0.0068892695045572605, 0.005824573761483487, 0.005404655461342722, -0.009529514326773958]
